# 各种layer的计算公式与效果

- [dropout](#dropout)
- [batchnorm](#batchnorm)

## dropout

**防止神经网络过拟合**的正则化方法，**提升神经网络的泛化性**。

- dropout对小数据防止过拟合有很好的效果,值一般设为0.5,小数据上dropout+sgd效果更好.
- 对于小数据集，即使使用BN，还是需要使用dropout。对于大数据集，使用BN就没必要使用dropout了

<font color=red>**训练时间会大大增长**</font>

概念：在神经网络训练中，**以一定的概率随机地丢弃一部分神经元**来**简化网络**的一项操作。本质上来说，dropout就是在正常的神经网络基础上给每一层的每一个神经元加了一道概率流程来**随机丢弃某些神经元以达到防止过拟合的目的**。

![dropout1示意图](../imgs/dropout1.jpg)

```python
from keras.layers import Dropout
x = Dropout(0.5)(x)
```



### dropout前向传播

原始前向传播过程：

$$z_i^{l+1}=\textbf{w}_i^{l+1}\textbf{y}^l+b_i^{l+1}$$

$$y_i^{l+1}=f(z_i^{l+1})$$

----

带dropout的前向传播过程：

$$r_j^l=Bernoulli(p)$$

$$\tilde{\textbf{y}}^l=\textbf{r}^l * \textbf{y}^l$$

$$z_i^{l+1}=\textbf{w}_i^{l+1}\tilde{\textbf{y}}^l+b_i^{l+1}$$

$$y_i^{l+1}=f(z_i^{l+1})$$

随机变量$r$是一个参数为$p$的bernolli二项分布

```python
# Inverted Dropout实现方法
# A1:上层输出 
# D1:生成随机数dropout向量
D1 = np.random.rand(A1.shape[0], A1.shape[1])
D1 = D1 < prob   # 小于prob的保留，其余的置零                    
A1 = np.multiply(D1, A1) # 屏蔽置零的神经元                     
A1 = A1 / prob # rescale，对保留的神经元扩大倍数
```
解释一：**为了保证神经网络在丢弃掉一些神经元之后总体信号强度不变和预测结果稳定，也有一种说法叫保证Bernoulli二项分布的数学期望不变，我们在Inverted Dropout方法中对dropout之后的做了除以p的rescale操作**。

解释二：当模型使用了dropout layer，训练的时候只有占比为 keep_prob 的隐藏层单元参与训练，那么在预测的时候，如果所有的隐藏层单元都需要参与进来，则**预测得到的结果相比训练时平均要大 1/keep_prob**，为了避免这种情况，就需要测试的时候将输出结果乘以 1/keep_prob 使下一层的输入规模保持不变。

而利用inverted dropout，我们可以在训练的时候直接将dropout后留下的权重扩大 1/keep_prob 倍，这样就可以使结果的scale保持不变，而在预测的时候也不用做额外的操作了，更方便一些。


Vanilla Dropout：当我们使用了dropout后，在模型训练阶段只有占比为p部分的神经元参与了训练，那么在预测阶段得到的结果会比实际平均要大1/p，所以在测试阶段我们需要将输出结果乘以p来保持输出规模不变。因为预测过程需要根据训练阶段所使用的dropout策略做调整，比较麻烦，所以一般不用。

### dropout反向传播

```python
dA1 = np.multiply(dA1, D1)      
dA1 = dA1 / prob  # 梯度计算时需要除以保留概率
```

另一个细节问题在于dropout有一种类似集成学习的boosting思想在里面。神经网络以drop_rate的概率丢弃某些神经元，**当进行多次训练时，因为随机性每次训练的都是不同的网络，dropout使得神经网络不依赖于某些独立的特征**，最后的结果也就是几次训练之后的**平均结果**，这些都使得神经网络具备**更好的泛化**性能，也正是dropout能够**防止过拟合**的主要原因。



## batchnorm

https://mp.weixin.qq.com/s/Ho2FwCYdQTf40MRUye_u9g

问题：随着网络加深，模型会越来越难以训练。

- sigmoid作为激活函数一个最大的问题会引起梯度消失现象，这使得神经网络难以更新权重。使用ReLu激活函数可以有效的缓解这一问题。

- 使用dropout来对神经网络进行简化，可以有效缓解神经网络的过拟合问题，对于深度网络的训练也有一定的帮助。

- ResNet使用残差块和skip connection来解决这个问题，使得深度加深时网络仍有较好的表现力。

- BN就是使得深度神经网络训练过程中**每一层网络输入都保持相同分布**，缓解梯度消失问题，提升训练速度和准确率，有轻微的正则化效果。用了bn就可以撤掉dropout，实验证明如此，有例外，看具体实验效果。

---

机器学习的一个重要假设就是IID(Independent Identically Distributed)假设，即**独立同分布假设**。所谓独立同分布，就是指<font color=red>**训练数据和测试数据是近似于同分布的**，如若**不然，机器学习模型就会很难有较好的泛化性能**</font>。

深度神经网络在训练过程中**每一层的输入并<font color=red>不满足独立同分布假设</font>，当叠加的网络层每一层的输入分布都发生变化时，这使得神经网络训练难以收敛**。这种<font color=red>神经网络隐藏层输入分布的不断变化的现象就叫Internal Covariate Shift(ICS)</font>。ICS问题正是导致深度神经网络难以训练的重要原因之一。

换句话说：

Covariance shift

   - 由于parameters发生变化，导致本层的activation发生变化
   - 当每层activation的分布都不相同或者差异较大时，
   - 训练效率会很低下
    
因此，解决方案就是同一每一层的activations的分布

---

数据标准化是将数据喂给机器学习模型之前一项重要的数据预处理技术，数据标准化也即将数据分布变换成均值为0，方差为1的标准正态分布，所以也叫0-1标准化。图像处理领域的数据标准化也叫白化(whiten)，当然，白化方法除了0-1标准化之外，还包括极大极小标准化方法。——联想：<font color=red>将这种白化操作推广到神经网络的每一个隐藏层</font>。



ICS问题导致深度神经网络训练**难以收敛，隐藏层输入分布逐渐向非线性激活函数取值区间的两端靠近**，比如说sigmoid函数的两端就是最大正值或者最小负值。这里说一下梯度饱和和梯度敏感的概念。当**取值位于sigmoid函数的两端时，即sigmoid取值接近0或1时，梯度接近于0，这时候就位于梯度饱和区**，也就是容易产生梯度消失的区域，相应的梯度敏感就是梯度计算远大于0，神经网络反向传播时每次都能使权重得到很好的更新。当梯度逐渐向这sigmoid两个端区域靠近时，就会产生梯度爆炸或者梯度消失问题，这也是深度神经网络难以训练的根本原因。


**BN将白化操作应用到每一个隐藏层，对每个隐藏层输入分布进行标准化变换，把每层的输入分布都强行拉回到均值为0方差为1的标准正态分布。这样一来，上一层的激活输出值(即当前层的激活输入值)就会落在非线性函数对输入的梯度敏感区，远离了原先的梯度饱和区，神经网络权重易于更新，训练速度相应加快。**

![bn位置示意图](../imgs/bn1.jpg)

由上图可见，BN对每个隐藏层的激活输出做标准化，位于隐藏层之后。

对于mini-batch SGD，batch_size=m,具体bn计算如下：

Input:Values of $x$ over mini-batch: $B={x_1...m}$;Parameters to be learned: $γ, β $

output：${y_i=BN_{\gamma, \beta}(x_i)}$

$\mu _B \leftarrow  \frac{1}{m}\sum_{i=1}^{m}x_i$  //mini-batch  mean

$\sigma _B^2 \leftarrow \frac{1}{m}\sum_{i=1}^{m}(x_i-\mu _B)^2$  //mini-batch variance

$\widehat{x}_i \leftarrow \frac{x_i-\mu _B}{\sqrt{\sigma_B^2 + \epsilon }}$  // normalize使得mean=0， variance= 1

$y_i \leftarrow \gamma\widehat{x}_i + \beta \equiv BN_{\gamma, \beta}(x_i)$  // scale and shift


关于为什么Batch Normalization 有了normalization, 即mean = 0， variance=1，还不够，为什么要有 $\gamma \beta$ ?

scale and shift的变换。从作者在论文中的表述看，认为*每一层都做BN之后可能会导致网络的表征能力下降*，所以这里增加两个调节参数(scale和shift)，对变换之后的结果进行反变换，**弥补网络的表征能力**。

以下代码来自 https://zhuanlan.zhihu.com/p/26138673
```python

def batchnorm_forward(x, gamma, beta, bn_param):
    """
    Forward pass for batch normalization.

    Input:
    - x: Data of shape (N, D)
    - gamma: Scale parameter of shape (D,)
    - beta: Shift paremeter of shape (D,)
    - bn_param: Dictionary with the following keys:
    - mode: 'train' or 'test'; required
    - eps: Constant for numeric stability
    - momentum: Constant for running mean / variance.
    - running_mean: Array of shape (D,) giving running mean of features
    - running_var Array of shape (D,) giving running variance of features

    Returns a tuple of:
    - out: of shape (N, D)
    - cache: A tuple of values needed in the backward pass
    """
    mode = bn_param['mode']
    eps = bn_param.get('eps', 1e-5)
    momentum = bn_param.get('momentum', 0.9)

    N, D = x.shape
    running_mean = bn_param.get('running_mean', np.zeros(D, dtype=x.dtype))
    running_var = bn_param.get('running_var', np.zeros(D, dtype=x.dtype))

    out, cache = None, None
    
    if mode == 'train':

        sample_mean = np.mean(x, axis=0)  # mini-batch mean
        sample_var = np.var(x, axis=0)  # mini-batch var
        out_ = (x - sample_mean) / np.sqrt(sample_var + eps)  # normalize
        
        running_mean = momentum * running_mean + (1 - momentum) * sample_mean  # for predict
        running_var = momentum * running_var + (1 - momentum) * sample_var  # for predict

        out = gamma * out_ + beta  # scale & shift
        cache = (out_, x, sample_var, sample_mean, eps, gamma, beta) # for grad

    elif mode == 'test':
        # 预测时没有mini-batch mean和var，以训练时记录的动态mean和var及训练得到的gamma、beta参数替代
        scale = gamma / np.sqrt(running_var + eps)
        out = x * scale + (beta - running_mean * scale)

    else:
        raise ValueError('Invalid forward batchnorm mode "%s"' % mode)

    # Store the updated running means back into bn_param
    bn_param['running_mean'] = running_mean
    bn_param['running_var'] = running_var

    return out, cache
```

## conv

## rnn